<a href="https://colab.research.google.com/github/Hiji1023/dl_network_cla_/blob/main/CH04_07_SCCE_and_CCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**4-4: Sparse Categorical Cross Entropy**#

##**Code4-4-1: SCCE Calcualtion**##

In [2]:
import tensorflow as tf

from tensorflow.keras.losses import SparseCategoricalCrossentropy

# batch_size = row의 수, n_class = column의 수
# x = 16 by 5 matrix
batch_size, n_class = 16,5

#predictions = y_hat,16 by 5 matrix, 0~1사이의 값
#마지막 뉴런의 갯수 = class의 갯수
predictions = tf.random.uniform(shape=(batch_size, n_class), minval=0, maxval=1, dtype=tf.float32)

# 행끼리의 합 구한 뒤, reshape해서 column vector로 만들어 줌 
pred_sum = tf.reshape(tf.reduce_sum(predictions, axis = 1), (-1,1))
predictions = predictions / pred_sum
# labels = y
labels = tf.random.uniform(shape=(batch_size, ),minval = 0, maxval = n_class, dtype=tf.int32)
loss_object = SparseCategoricalCrossentropy()
loss = loss_object(labels, predictions)
print(loss.numpy())

ce = 0
for label, prediction in zip(labels, predictions):
  ce += -tf.math.log(prediction[label])
ce /= batch_size
print(ce.numpy())

# loss = ce

1.8806874
1.8806875


##**Code4-4-1: SCCE with Model/Dataset**##

In [16]:
import tensorflow as tf

from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

N, n_feature = 100, 2
n_class = 5

X = tf.zeros(shape=(0, n_feature))
Y = tf.zeros(shape=(0,1), dtype=tf.int32)

for class_idx in range(n_class):
  center = tf. random.uniform(minval=-15, maxval=15, shape=(2, ))

  x1 = center[0] + tf.random.normal(shape=(N, 1))
  x2 = center[1] + tf.random.normal(shape=(N, 1))

  x = tf.concat((x1, x2), axis=1)
  y = class_idx*tf.ones(shape=(N, 1), dtype=tf.int32)

  X = tf.concat((X, x), axis = 0)
  Y = tf.concat((Y, y), axis = 0)

# tensor에서 만들거니깐, (X,Y) -> tuple로 넣어줌
# 열기준으로 slice -> dataset 2개 , dataset당 x,y는 batch_size개 들어있다

dataset = tf.data.Dataset.from_tensor_slices((X,Y))
print(dataset)
dataset = dataset.batch(batch_size)

model = Dense(units=n_class, activation='softmax')
loss_object = SparseCategoricalCrossentropy()

# loss가 16 * 2 개 나와야함
for x,y in dataset:
  predictions = model(x)
  loss = loss_object(y, predictions)
  print(loss.numpy())

<TensorSliceDataset element_spec=(TensorSpec(shape=(2,), dtype=tf.float32, name=None), TensorSpec(shape=(1,), dtype=tf.int32, name=None))>
7.0570326
7.0201874
6.9119596
6.921844
6.9838066
6.9927797
10.15341
11.246576
11.018984
11.169491
10.913704
11.154956
5.610979
0.09014476
0.07343918
0.075236864
0.06897332
0.080975465
3.604432
13.280009
13.460102
13.393946
13.663402
13.775893
13.933668
11.471234
11.71542
11.487019
12.066933
11.396588
12.009353
11.740828


#**4-5: Categorical Cross Entropy**##

##**Code4-5-1: CCE Calcualtion**##

In [ ]:
import tensorflow as tf

from tensorflow.keras.losses import CategoricalCrossentropy

# x = 16 by 5 matrix
batch_size, n_class = 16, 5

#0 <= y_hat = predictons <= 1 
#마지막 뉴런의 갯수 = class의 갯수
predictions = tf.random.uniform(shape=(batch_size, n_class), minval=0, maxval=1, dtype=tf.float32)

# 행끼리의 합 구한 뒤, reshape해서 column vector로 만들어 줌 
# pred_sum = 16 by 1 matrix
pred_sum = tf.reshape(tf.reduce_sum(predictions, axis = 1), (-1,1))
predictions = predictions / pred_sum
print(predictions.shape)
# labels = y
labels = tf.random.uniform(shape=(batch_size, ),minval = 0, maxval = n_class, dtype=tf.int32)
# one_hot(indices, depth), labels by n_class matrix
labels = tf.one_hot(labels, n_class)

loss_object = CategoricalCrossentropy()
loss = loss_object(labels, predictions)

print("CCE(Tensroflow): ",loss.numpy())

cce_man = tf.reduce_mean(tf.reduce_sum(-labels*tf.math.log(predictions), axis = 1))
print("CCE(Manual): ",cce_man.numpy())

(16, 5)
CCE(Tensroflow):  1.4721774
CCE(Manual):  1.4721774


##**Code4-5-2: CCE with Model/Dataset**##

In [23]:
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy

N, n_feature = 8, 2
n_class = 5

X = tf.zeros(shape=(0, n_feature))
Y = tf.zeros(shape=(0,), dtype=tf.int32)

for class_idx in range(n_class):
  center = tf.random.uniform(minval=-15, maxval=15, shape=(2, ))

  x1 = center[0] + tf.random.normal(shape=(N, 1))
  x2 = center[1] + tf.random.normal(shape=(N, 1))

  #Concatenates tensors along one dimension= tf.concat()
  x = tf.concat((x1, x2), axis=1)
  #Creates a tensor with all elements set to one (1) = ones()
  y = class_idx*tf.ones(shape=(N, ), dtype=tf.int32)

  X = tf.concat((X, x), axis = 0)
  Y = tf.concat((Y, y), axis = 0)

#one_hot() = Returns a one-hot tensor, depth = 클래스 갯수
Y = tf.one_hot(Y, depth=n_class, dtype=tf.int32)

# 열기준으로 slice -> dataset 2개 , dataset당 x,y는 batch_size개 들어있다 
dataset = tf.data.Dataset.from_tensor_slices((X,Y))
dataset = dataset.batch(batch_size)

model = Dense(units=n_class, activation='softmax')
loss_object = CategoricalCrossentropy()

for x,y in dataset:
  predictions = model(x)
  loss = loss_object(y, predictions)
  print(loss.numpy())


8.960773
4.633885
0.11723381
